In [10]:
%cd ..
%rm -rf no-time-to-backprop

/content


In [1]:
!ls
!nvidia-smi

sample_data
Mon Dec 22 18:18:59 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   38C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------

In [2]:
import os
os.environ["XLA_PYTHON_CLIENT_PREALLOCATE"] = "false"
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"] = "0.80"

!git clone https://github.com/Francis2002/no-time-to-backprop.git
%cd no-time-to-backprop

Cloning into 'no-time-to-backprop'...
remote: Enumerating objects: 281, done.
remote: Counting objects: 100% (143/143), done.
remote: Compressing objects: 100% (90/90), done.
remote: Total 281 (delta 88), reused 91 (delta 43), pack-reused 138 (from 1)
Receiving objects: 100% (281/281), 97.87 MiB | 15.88 MiB/s, done.
Resolving deltas: 100% (157/157), done.
/content/no-time-to-backprop


In [3]:
!python -V
!ls

Python 3.12.12
constraints.txt			 requirements-linux-cuda.txt  run_all_toy.sh
hpt.py				 requirements-macos-cpu.txt   run_on_gpu.ipynb
plotting_scripts		 requirements.mac.txt	      run_toy_task.py
requirements-base.txt		 requirements.txt	      test_rec.py
requirements.from_linux_env.txt  run_all_benchmarks.sh	      tgap


In [4]:
%pip install -r requirements-linux-cuda.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.8/52.8 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.8/135.8 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.0/101.0 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 488.2/488.2 kB 40.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 131.8 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.6/16.6 MB 102.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 404.7/404.7 kB 31.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 139.0 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 144.2 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 243.4/243.4 kB 22.4 MB/s eta 0:00:00
   ━━━

In [5]:
import jax
import jaxlib
import sys

print(f"jax: {jax.__version__}")
print(f"jaxlib: {jaxlib.__version__}")
print(f"executable: {sys.executable}")
print(f"devices: {jax.devices()}")


jax: 0.8.1
jaxlib: 0.8.1
executable: /usr/bin/python3
devices: [CudaDevice(id=0)]


In [7]:
!python3 run_toy_task.py -m ONLINE -a ZUC --dataset mooc --task link_classification

[*] Results will be stored in /content/no-time-to-backprop/results_for_rotational_vs_none/mooc_ONLINE
[*] d_model set to 64
Running iteration: 0
[*] Output file /content/no-time-to-backprop/tgap/data/npzs/mooc_stream.npz already exists. Using existing file.
object address  : 0x7bd2cbad2f20
object refcount : 3
object type     : 0xa2a4e0
object type name: KeyboardInterrupt
object repr     : KeyboardInterrupt()
lost sys.stderr
^C


In [6]:
# Here we run experiments
!python3 run_toy_task.py -m ONLINE -a ZUC --dataset mooc --batching_strategy rearranged --num_epochs 100 --dedupe --hpt optuna --hpt_config ./tgap/benchmarks/hpt_config_example.json


[*] Results will be stored in /content/no-time-to-backprop/results_for_benchmarks/mooc_ONLINE
[*] d_model set to 16
[I 2025-12-22 18:24:15,086] A new study created in memory with name: no-name-0810d407-b766-48f1-99c1-1224828a7314

[*] ----------------------------- Iteration: 0 -----------------------------

[*] Trial: <optuna.trial._trial.Trial object at 0x78409ace8740>
[*] HPT: {'memory': 5, 'seed': 43, 'num_layers': 3, 'state_size': 64, 'd_model_factor': 0.5, 'dropout': 0.06525516552683015, 'pos_weight': 1.0, 'batch_size': 64, 'n_accumulation_steps': 13, 'learning_rate': 0.000906841944816923, 'rec_learning_factor': 0.5, 'beta1': 0.9335020410221113, 'beta2': 0.9550937756942832, 'weight_decay': 0.00037865231131024233}
[*] Npz file /content/no-time-to-backprop/tgap/data/npzs/mooc_stream.npz already exists with matching meta. Using existing file.
W1222 18:24:15.437358    1950 cuda_executor.cc:1802] GPU interconnect information not available: INTERNAL: NVML doesn't support extracting fab